In [2]:
import numpy as np
import sys
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
import pandas as pd
import GPy
import math
import matplotlib as mpl
import mpl_toolkits
import matplotlib.pyplot as plt
import scipy
import netCDF4 as nc
import glob
import scipy.io
from cycler import cycler
from scipy.interpolate import interp1d

Load up the MICI data

In [8]:
df2300_26 = pd.read_excel('./Data/41586_2021_3427_MOESM6_ESM.xlsx', 'RCP2.6 sea level equivalent (m)', engine='openpyxl')
df2300_26 = df2300_26.iloc[:,:110]

df2300_45 = pd.read_excel('./Data/41586_2021_3427_MOESM6_ESM.xlsx', 'RCP4.5 sea level equivalent (m)', engine='openpyxl')
df2300_45 = df2300_45.iloc[:,:110]

Transform the data so baseline is 2020, as all of our data will be.

In [5]:
df2300_26.iloc[:,1:] = df2300_26.iloc[:,1:] - df2300_26.iloc[20,1:]
df2300_45.iloc[:,1:] = df2300_45.iloc[:,1:] - df2300_45.iloc[20,1:]

Let's look at the last 50 years of simulations, and try fitting a linear model

In [9]:
years = np.linspace(2301, 2500, 200)
coef_26 = np.zeros((df2300_26.shape[1]-1,2))

for i in np.arange(df2300_26.shape[1])[1:]:
    regr = linear_model.LinearRegression()
    regr.fit(df2300_26.iloc[250:301,0].to_frame(), df2300_26.iloc[250:301,i].to_frame())
    coef_26[i-1,0] = regr.coef_
    coef_26[i-1,1] = regr.intercept_

coef_45 = np.zeros((df2300_45.shape[1]-1,2))

for i in np.arange(df2300_45.shape[1])[1:]:
    regr = linear_model.LinearRegression()
    regr.fit(df2300_45.iloc[250:301,0].to_frame(), df2300_45.iloc[250:301,i].to_frame())
    coef_45[i-1,0] = regr.coef_
    coef_45[i-1,1] = regr.intercept_



According to Meinshausen et al (2020), CO2 concentrations will not rise linearly after 2300 but will instead decrease slightly. A linear prediction therefore may be unreasonable to calculate. Instead we consider two extremes, a linear increase or a near flat increase in sea level rise, calculated as 10% of the linear model gradient. We sample inbetween each of these extremes using a Uniform distribution, randomly assigning the gradients to each simulation and using the 2300 prediction as the intercept. 

In [10]:
np.random.seed(0)
years = np.linspace(2301, 2500, 200)
years = np.reshape(years, (200,1))
years2 = np.linspace(1, 200, 200)
pp_26 = np.zeros((200,coef_26.shape[0]))
grad_26 = np.zeros(coef_26.shape[0])
for i in np.arange(coef_26.shape[0]):
    grad_26[i] = np.random.uniform(0.1*coef_26[i,0], coef_26[i,0])
    for j in np.arange(200):
        pp_26[j,i] = grad_26[i]*years2[j] + df2300_26.iloc[300,i+1]
    
pp_45 = np.zeros((200,coef_45.shape[0]))
grad_45 = np.zeros(coef_45.shape[0])
for i in np.arange(coef_45.shape[0]):
    grad_45[i] = np.random.uniform(0.1*coef_45[i,0], coef_45[i,0])
    for j in np.arange(200):
        pp_45[j,i] = grad_45[i]*years2[j] + df2300_45.iloc[300,i+1]


In [11]:
DeConto_26_2500 = np.concatenate((df2300_26.iloc[:,1:],pp_26))
DeConto_45_2500 = np.concatenate((df2300_45.iloc[0:301,1:],pp_45))
DeConto_years = np.concatenate((df2300_26['Year'],np.concatenate(years)))

DeConto_26_csv = np.concatenate((np.reshape(DeConto_years, (501,1)), DeConto_26_2500),axis=1)
DeConto_45_csv = np.concatenate((np.reshape(DeConto_years, (501,1)), DeConto_45_2500),axis=1)

#np.savetxt('./Created_Data/DeConto_2500_RCP26.csv', DeConto_26_csv[20:,:], delimiter=',')
#np.savetxt('./Created_Data/DeConto_2500_RCP45.csv', DeConto_45_csv[20:,:], delimiter=',')

In [12]:
pp = np.array([0.05, 0.17, 0.5, 0.83, 0.95])

In [13]:
quants = np.array([0.   , 0.001, 0.005, 0.01 , 0.02 , 0.03 , 0.04 , 0.05 ,
                   0.06 , 0.07 , 0.08 , 0.09 , 0.1  , 0.11 , 0.12 , 0.13 ,
                   0.14 , 0.15 , 0.16 , 0.167, 0.17 , 0.18 , 0.19 , 0.2  ,
                   0.21 , 0.22 , 0.23 , 0.24 , 0.25 , 0.26 , 0.27 , 0.28 ,
                   0.29 , 0.3  , 0.31 , 0.32 , 0.33 , 0.34 , 0.35 , 0.36 ,
                   0.37 , 0.38 , 0.39 , 0.4  , 0.41 , 0.42 , 0.43 , 0.44 ,
                   0.45 , 0.46 , 0.47 , 0.48 , 0.49 , 0.5  , 0.51 , 0.52 ,
                   0.53 , 0.54 , 0.55 , 0.56 , 0.57 , 0.58 , 0.59 , 0.6  ,
                   0.61 , 0.62 , 0.63 , 0.64 , 0.65 , 0.66 , 0.67 , 0.68 ,
                   0.69 , 0.7  , 0.71 , 0.72 , 0.73 , 0.74 , 0.75 , 0.76 ,
                   0.77 , 0.78 , 0.79 , 0.8  , 0.81 , 0.82 , 0.83 , 0.833,
                   0.84 , 0.85 , 0.86 , 0.87 , 0.88 , 0.89 , 0.9  , 0.91 ,
                   0.92 , 0.93 , 0.94 , 0.95 , 0.96 , 0.97 , 0.98 , 0.99 ,
                   0.995, 0.999, 1. ])

Now have the Bulthuis et al data.

In [14]:
bult_26_m1 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m1/RCP26/response.mat', squeeze_me = True)
bult_45_m1 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m1/RCP45/response.mat', squeeze_me = True)

bult_26_m2 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m2/RCP26/response.mat', squeeze_me = True)
bult_45_m2 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m2/RCP45/response.mat', squeeze_me = True)

bult_26_m2t = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m2Tsai/RCP26/response.mat', squeeze_me = True)
bult_45_m2t = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m2Tsai/RCP45/response.mat', squeeze_me = True)

bult_26_m3 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m3/RCP26/response.mat', squeeze_me = True)
bult_45_m3 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m3/RCP45/response.mat', squeeze_me = True)

bult_26_m5 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m5/RCP26/response.mat', squeeze_me = True)
bult_45_m5 = scipy.io.loadmat('./Data/Data_Bulthuis_ensemble/m5/RCP45/response.mat', squeeze_me = True)

bult_26 = np.vstack((bult_26_m1['data'], bult_26_m2['data'], bult_26_m2t['data'], bult_26_m3['data'], bult_26_m5['data']))
bult_45 = np.vstack((bult_45_m1['data'], bult_45_m2['data'], bult_45_m2t['data'], bult_45_m3['data'], bult_45_m5['data']))

Transform data so baseline is 2020, then interpolate to annual.

In [15]:
bult_26 = bult_26 - bult_26[:,5:6]
bult_45 = bult_45 - bult_45[:,5:6]


annual_bult_26 = np.zeros((DeConto_years.shape[0],bult_26.shape[0]))
for i in np.arange(bult_26.shape[0]):
    f126 = interp1d(bult_26_m1['time'][0:102], bult_26[i,0:102], fill_value="extrapolate")
    annual_bult_26[:,i] = f126(DeConto_years)

annual_bult_45 = np.zeros((DeConto_years.shape[0],bult_45.shape[0]))
for i in np.arange(bult_45.shape[0]):
    f245 = interp1d(bult_45_m1['time'][0:102], bult_45[i,0:102], fill_value="extrapolate")
    annual_bult_45[:,i] = f245(DeConto_years)

bult_26_csv = np.concatenate((np.reshape(DeConto_years, (501,1)), annual_bult_26),axis=1)
bult_45_csv = np.concatenate((np.reshape(DeConto_years, (501,1)), annual_bult_45),axis=1)

#np.savetxt('./Created_Data/Bulthuis_2500_RCP26.csv', bult_26_csv[20:,:], delimiter=',')
#np.savetxt('./Created_Data/Bulthuis_2500_RCP45.csv', bult_45_csv[20:,:], delimiter=',')

And now the Lowry et al data

In [16]:
Lowry_26 = pd.read_csv('./Data/Lowry_et_al/emulator_runs/emulator_runs_rcp26.csv')

Lowry_45 = pd.read_csv('./Data/Lowry_et_al/emulator_runs/emulator_runs_rcp45.csv')


Transform the data so baseline is 2020

In [17]:
Lowry_26.iloc[:,2:] = Lowry_26.iloc[:,2:] - Lowry_26.iloc[50,2:]
Lowry_45.iloc[:,2:] = Lowry_45.iloc[:,2:] - Lowry_45.iloc[50,2:]

Let's extrapolate.

In [18]:
coef_Lowry_26 = np.zeros((Lowry_26.shape[1]-2,2))

for i in np.arange(Lowry_26.shape[1])[2:]:
    regr = linear_model.LinearRegression()
    regr.fit(Lowry_26.iloc[280:331,0].to_frame(), Lowry_26.iloc[280:331,i].to_frame())
    coef_Lowry_26[i-2,0] = regr.coef_
    coef_Lowry_26[i-2,1] = regr.intercept_


In [19]:
np.random.seed(0)
years2 = np.linspace(1, 200, 200)
np.random.seed(0)
pp_Lowry_26 = np.zeros((200,coef_Lowry_26.shape[0]))
grad_Lowry_26 = np.zeros(coef_Lowry_26.shape[0])
for i in np.arange(coef_Lowry_26.shape[0]):
    grad_Lowry_26[i] = np.random.uniform(0.1*coef_Lowry_26[i,0], coef_Lowry_26[i,0])
    for j in np.arange(200):
        pp_Lowry_26[j,i] = grad_Lowry_26[i]*years2[j] + Lowry_26.iloc[330,i+2]

In [20]:
coef_Lowry_45 = np.zeros((Lowry_45.shape[1]-2,2))

for i in np.arange(Lowry_45.shape[1])[2:]:
    regr = linear_model.LinearRegression()
    regr.fit(Lowry_45.iloc[280:331,0].to_frame(), Lowry_45.iloc[280:331,i].to_frame())
    coef_Lowry_45[i-2,0] = regr.coef_
    coef_Lowry_45[i-2,1] = regr.intercept_


In [21]:
np.random.seed(0)
pp_Lowry_45 = np.zeros((200,coef_Lowry_45.shape[0]))
grad_Lowry_45 = np.zeros(coef_Lowry_45.shape[0])
for i in np.arange(coef_Lowry_45.shape[0]):
    grad_Lowry_45[i] = np.random.uniform(0.1*coef_Lowry_45[i,0], coef_Lowry_45[i,0])
    for j in np.arange(200):
        pp_Lowry_45[j,i] = grad_Lowry_45[i]*years2[j] + Lowry_45.iloc[330,i+2]


In [22]:
years2 = np.reshape(years2, (200,1))
Lowry_26_2500 = np.concatenate((Lowry_26.iloc[:,2:],pp_Lowry_26))
Lowry_45_2500 = np.concatenate((Lowry_45.iloc[:,2:],pp_Lowry_45))
Lowry_years = np.concatenate((Lowry_26['year'],np.concatenate(years2+2300)))

Lowry_26_csv = np.concatenate((np.reshape(Lowry_years, (531,1)), Lowry_26_2500),axis=1)
Lowry_45_csv = np.concatenate((np.reshape(Lowry_years, (531,1)), Lowry_45_2500),axis=1)

#np.savetxt('./Created_Data/Lowry_2500_RCP26.csv', Lowry_26_csv[50:,:], delimiter=',')
#np.savetxt('./Created_Data/Lowry_2500_RCP45.csv', Lowry_45_csv[50:,:], delimiter=',')

Let's look at the LARMIP data.

In [23]:
fn = './Data/fortamsin_221025/icesheets-ipccar6-larmipicesheet-ssp126_TOT_globalsl.nc'
ds_LAR126 = nc.Dataset(fn)
slc_126 = ds_LAR126['sea_level_change'][:,:,:]
slc_126 = np.squeeze(slc_126)
slc_126 = slc_126.transpose()
slc_126 = slc_126 - slc_126[0,:]

years_TOT = ds_LAR126['years'][:]
years_TOT

fn = './Data/fortamsin_221025/icesheets-ipccar6-larmipicesheet-ssp245_TOT_globalsl.nc'
ds_LAR245 = nc.Dataset(fn)
slc_245 = ds_LAR245['sea_level_change'][:,:,:]
slc_245 = np.squeeze(slc_245)
slc_245 = slc_245.transpose()
slc_245 = slc_245 - slc_245[0,:]

q = [7, 19, 53, 87, 99]

Need to interpolate to annual data, and then extrapolate.

In [24]:
yearsnew = np.linspace(2020, 2300, num=281, endpoint=True)
yearsnew = np.reshape(yearsnew, (281,))

np.random.seed(0)
annualLAR126 = np.zeros((yearsnew.shape[0],slc_126.shape[1]))
LAR126_2500 = np.zeros((years.shape[0],slc_126.shape[1]))

for i in np.arange(slc_126.shape[1]):
    f126 = interp1d(years_TOT, slc_126[:,i], fill_value="extrapolate")
    annualLAR126[:,i] = f126(yearsnew)

coef_LAR_126 = np.zeros((annualLAR126.shape[1],2))

for i in np.arange(annualLAR126.shape[1]):
    regr = linear_model.LinearRegression()
    regr.fit(yearsnew[231:281].reshape((50,1)), annualLAR126[231:281,i].reshape((50,1)))
    coef_LAR_126[i,0] = regr.coef_
    coef_LAR_126[i,1] = regr.intercept_

pp_LAR_126 = np.zeros((200,coef_LAR_126.shape[0]))
grad_LAR_126 = np.zeros(coef_LAR_126.shape[0])
for i in np.arange(coef_LAR_126.shape[0]):
    grad_LAR_126[i] = np.random.uniform(0.1*coef_LAR_126[i,0], coef_LAR_126[i,0])
    for j in np.arange(200):
        pp_LAR_126[j,i] = grad_LAR_126[i]*(j+1) + annualLAR126[280,i]       

        
annualLAR245 = np.zeros((yearsnew.shape[0],slc_245.shape[1]))
LAR245_2500 = np.zeros((years.shape[0],slc_245.shape[1]))

for i in np.arange(slc_245.shape[1]):
    f245 = interp1d(years_TOT, slc_245[:,i], fill_value="extrapolate")
    annualLAR245[:,i] = f245(yearsnew)

coef_LAR_245 = np.zeros((annualLAR245.shape[1],2))

for i in np.arange(annualLAR245.shape[1]):
    regr = linear_model.LinearRegression()
    regr.fit(yearsnew[231:281].reshape((50,1)), annualLAR245[231:281,i].reshape((50,1)))
    coef_LAR_245[i,0] = regr.coef_
    coef_LAR_245[i,1] = regr.intercept_

pp_LAR_245 = np.zeros((200,coef_LAR_245.shape[0]))
grad_LAR_245 = np.zeros(coef_LAR_245.shape[0])
for i in np.arange(coef_LAR_245.shape[0]):
    grad_LAR_245[i] = np.random.uniform(0.1*coef_LAR_245[i,0], coef_LAR_245[i,0])
    for j in np.arange(200):
        pp_LAR_245[j,i] = grad_LAR_245[i]*(j+1) + annualLAR245[280,i]   


LAR_years = np.append(yearsnew,years)
LAR_126 = np.concatenate((annualLAR126,pp_LAR_126))/1000
LAR_245 = np.concatenate((annualLAR245,pp_LAR_245))/1000

LAR_126_csv = np.concatenate((np.reshape(LAR_years, (481,1)), LAR_126),axis=1)
LAR_245_csv = np.concatenate((np.reshape(LAR_years, (481,1)), LAR_245),axis=1)

#np.savetxt('./Created_Data/LARMIP_2500_SSP126.csv', LAR_126_csv, delimiter=',')
#np.savetxt('./Created_Data/LARMIP_2500_SSP245.csv', LAR_245_csv, delimiter=',')

What is going on in Greenland?

Look at Aschwanden et al first.

Load data, which is stored in kg. Convert to SLE.

In [25]:
files45 = glob.glob("./Data/resource_map_doi_10_18739_A2222R58F/data/*rcp_45*")
files26 = glob.glob("./Data/resource_map_doi_10_18739_A2222R58F/data/*rcp_26*")

kg45 = np.zeros((1000, len(files45)))

for i in np.arange(len(files45)):
    fn = files45[i]
    ds = nc.Dataset(fn)
    kg45[:,i] = ds['limnsw'][:]
    
time = ds['time'][:]
time = np.reshape(time, (1000,1))

kg45 = np.append(np.rint(time/31536000 + 2008), kg45, axis=1)


kg26 = np.zeros((1000, len(files26)))

for i in np.arange(len(files26)):
    fn = files26[i]
    ds = nc.Dataset(fn)
    kg26[:,i] = ds['limnsw'][:]
    
kg26 = np.append(np.rint(time/31536000 + 2008), kg26, axis=1)

grisyears = np.array(np.linspace(2008, 3007, num=1000))
grisyears = np.reshape(grisyears, (1000,1))

SLE26 = -1*(kg26[:,1:] - kg26[0,1:])/(365*(10**13))
SLE26 = SLE26[:,::-1]
SLE26 = np.append(grisyears, SLE26, axis=1)

SLE45 = -1*(kg45[:,1:] - kg45[0,1:])/(365*(10**13))
SLE45 = SLE45[:,::-1]
SLE45 = np.append(grisyears, SLE45, axis=1)

SLE26[:, 1:] = (SLE26[:,1:] - SLE26[12,1:])/100
SLE45[:, 1:] = (SLE45[:,1:] - SLE45[12,1:])/100

#np.savetxt('./Created_Data/Aschwanden_2500_RCP26.csv', SLE26[12:493,:], delimiter=',')
#np.savetxt('./Created_Data/Aschwanden_2500_RCP45.csv', SLE45[12:493,:], delimiter=',')

AR6 Greenland data

In [26]:
fn = './Data/AR6_GSAT,_Greenland_and_Antarctic_data/AR6_FittedISMIP_GIS_LARMIP_AIS/icesheets-FittedISMIP-icesheets-ssp126_GIS_globalsl.nc'
ds_26 = nc.Dataset(fn)
print(ds_26)
gris26 = ds_26['sea_level_change'][:,:,:]
gris26 = np.squeeze(gris26)
gris26 = gris26.transpose()

years_ar6 = ds_26['years'][:]

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: Global SLR contribution from GIS according to FittedISMIP icesheets workflow
    history: Created Fri Jun 25 15:03:59 2021
    source: FACTS: icesheets-FittedISMIP-icesheets-ssp126
    scenario: ssp126
    baseyear: 2005
    dimensions(sizes): years(29), samples(20000), locations(1)
    variables(dimensions): int32 years(years), int64 samples(samples), int64 locations(locations), float32 lat(locations), float32 lon(locations), int16 sea_level_change(samples, years, locations)
    groups: 


In [27]:
fn = './Data/AR6_GSAT,_Greenland_and_Antarctic_data/AR6_FittedISMIP_GIS_LARMIP_AIS/icesheets-FittedISMIP-icesheets-ssp245_GIS_globalsl.nc'
ds_45 = nc.Dataset(fn)
print(ds_45)
gris45 = ds_45['sea_level_change'][:,:,:]
gris45 = np.squeeze(gris45)
gris45 = gris45.transpose()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: Global SLR contribution from GIS according to FittedISMIP icesheets workflow
    history: Created Fri Jun 25 15:04:06 2021
    source: FACTS: icesheets-FittedISMIP-icesheets-ssp245
    scenario: ssp245
    baseyear: 2005
    dimensions(sizes): years(29), samples(20000), locations(1)
    variables(dimensions): int32 years(years), int64 samples(samples), int64 locations(locations), float32 lat(locations), float32 lon(locations), int16 sea_level_change(samples, years, locations)
    groups: 


Change baseline to 2020.

In [28]:
gris26 = gris26[:,:] - gris26[0,:]

In [29]:
gris45 = gris45[:,:] - gris45[0,:]

Interpolate into annual data, then extrapolate final fifty years using uniform choice of gradient.

In [30]:
np.random.seed(0)
xnew_ar6 = np.linspace(2301, 2500, num=200, endpoint=True)
yearsnew_ar6 = np.linspace(2020, 2300, num=281, endpoint=True)
annual26 = np.zeros((yearsnew_ar6.shape[0],gris26.shape[1]))
gris26_2500 = np.zeros((xnew_ar6.shape[0],gris26.shape[1]))

for i in np.arange(gris26.shape[1]):
    f26 = interp1d(years_ar6, gris26[:,i], fill_value="extrapolate")
    annual26[:,i] = f26(yearsnew_ar6)

coef_GrIS_26 = np.zeros((annual26.shape[1],2))

for i in np.arange(annual26.shape[1]):
    regr = linear_model.LinearRegression()
    regr.fit(yearsnew_ar6[81:131].reshape((50,1)), annual26[81:131,i].reshape((50,1)))
    coef_GrIS_26[i,0] = regr.coef_
    coef_GrIS_26[i,1] = regr.intercept_

pp_GrIS_26 = np.zeros((xnew_ar6.shape[0],coef_GrIS_26.shape[0])) 
grad_GrIS_26 = np.zeros(coef_GrIS_26.shape[0])
for i in np.arange(coef_GrIS_26.shape[0]):
    grad_GrIS_26[i] = np.random.uniform(0.1*coef_GrIS_26[i,0], coef_GrIS_26[i,0])
    for j in np.arange(200):
        pp_GrIS_26[j,i] = grad_GrIS_26[i]*(j+1) + annual26[280,i]   

annual45 = np.zeros((yearsnew_ar6.shape[0],gris45.shape[1]))
gris45_2500 = np.zeros((xnew_ar6.shape[0],gris45.shape[1]))

for i in np.arange(gris45.shape[1]):
    f45 = interp1d(years_ar6, gris45[:,i], fill_value="extrapolate")
    annual45[:,i] = f45(yearsnew_ar6)

coef_GrIS_45 = np.zeros((annual45.shape[1],2))

for i in np.arange(annual45.shape[1]):
    regr = linear_model.LinearRegression()
    regr.fit(yearsnew_ar6[81:131].reshape((50,1)), annual45[81:131,i].reshape((50,1)))
    coef_GrIS_45[i,0] = regr.coef_
    coef_GrIS_45[i,1] = regr.intercept_

pp_GrIS_45 = np.zeros((xnew_ar6.shape[0],coef_GrIS_45.shape[0])) 
grad_GrIS_45 = np.zeros(coef_GrIS_45.shape[0])
for i in np.arange(coef_GrIS_45.shape[0]):
    grad_GrIS_45[i] = np.random.uniform(0.1*coef_GrIS_45[i,0], coef_GrIS_45[i,0])
    for j in np.arange(200):
        pp_GrIS_45[j,i] = grad_GrIS_45[i]*(j+1) + annual45[280,i]   


gris_years = np.append(yearsnew_ar6,xnew_ar6)
gris_26 = np.concatenate((annual26,pp_GrIS_26))
gris_45 = np.concatenate((annual45,pp_GrIS_45))

In [31]:
gris_126_csv = np.concatenate((np.reshape(gris_years, (481,1)), gris_26/1000),axis=1)
gris_245_csv = np.concatenate((np.reshape(gris_years, (481,1)), gris_45/1000),axis=1)
#np.savetxt('./Created_Data/GrIS_AR6_2500_SSP126.csv', gris_126_csv, delimiter=',')
#np.savetxt('./Created_Data/GrIS_AR6_2500_SSP245.csv', gris_245_csv, delimiter=',')

Now let's work with the glaciers data

In [34]:
fn126 = './Data/glaciers-ar5-glaciersgmip2/glaciers-ar5-glaciersgmip2-ssp126_globalsl.nc'
ds126 = nc.Dataset(fn126)

fn245 = './Data/glaciers-ar5-glaciersgmip2/glaciers-ar5-glaciersgmip2-ssp245_globalsl.nc'
ds245 = nc.Dataset(fn245)

years_glac = ds126['years'][:]
years_glac

quant = ds126['quantiles'][:]
quant

masked_array(data=[0.   , 0.001, 0.005, 0.01 , 0.02 , 0.03 , 0.04 , 0.05 ,
                   0.06 , 0.07 , 0.08 , 0.09 , 0.1  , 0.11 , 0.12 , 0.13 ,
                   0.14 , 0.15 , 0.16 , 0.167, 0.17 , 0.18 , 0.19 , 0.2  ,
                   0.21 , 0.22 , 0.23 , 0.24 , 0.25 , 0.26 , 0.27 , 0.28 ,
                   0.29 , 0.3  , 0.31 , 0.32 , 0.33 , 0.34 , 0.35 , 0.36 ,
                   0.37 , 0.38 , 0.39 , 0.4  , 0.41 , 0.42 , 0.43 , 0.44 ,
                   0.45 , 0.46 , 0.47 , 0.48 , 0.49 , 0.5  , 0.51 , 0.52 ,
                   0.53 , 0.54 , 0.55 , 0.56 , 0.57 , 0.58 , 0.59 , 0.6  ,
                   0.61 , 0.62 , 0.63 , 0.64 , 0.65 , 0.66 , 0.67 , 0.68 ,
                   0.69 , 0.7  , 0.71 , 0.72 , 0.73 , 0.74 , 0.75 , 0.76 ,
                   0.77 , 0.78 , 0.79 , 0.8  , 0.81 , 0.82 , 0.83 , 0.833,
                   0.84 , 0.85 , 0.86 , 0.87 , 0.88 , 0.89 , 0.9  , 0.91 ,
                   0.92 , 0.93 , 0.94 , 0.95 , 0.96 , 0.97 , 0.98 , 0.99 ,
                   0.995,

Load data and set baseline to 2020.

In [35]:
q = [7, 19, 53, 87, 99]
glac126 = ds126['sea_level_change'][:]
glac126 = np.squeeze(glac126)
glac126 = glac126.T
glac126 = glac126 - glac126[0,q[2]]

glac245 = ds245['sea_level_change'][:]
glac245 = np.squeeze(glac245)
glac245 = glac245.T
glac245 = glac245 - glac245[0,q[2]]

Interpolate to annual data, then extrapolate to 2500.

In [36]:
annual126 = np.zeros((yearsnew_ar6.shape[0],glac126.shape[1]))
glac126_2500 = np.zeros((xnew_ar6.shape[0],glac126.shape[1]))

for i in np.arange(glac126.shape[1]):
    f126 = interp1d(years_glac, glac126[:,i], fill_value="extrapolate")
    annual126[:,i] = f126(yearsnew_ar6)
    glac126_2500[:,i] = f126(xnew_ar6)

annual245 = np.zeros((yearsnew.shape[0],glac245.shape[1]))
glac245_2500 = np.zeros((xnew_ar6.shape[0],glac245.shape[1]))

for i in np.arange(glac245.shape[1]):
    f245 = interp1d(years_glac, glac245[:,i], fill_value="extrapolate")
    annual245[:,i] = f245(yearsnew_ar6)
    glac245_2500[:,i] = f245(xnew_ar6)

yearsnew = np.reshape(yearsnew, (281,1))

Set a cap on glaciers contribution to sea level. This is set as the max value in the data.

In [37]:
for i in np.arange(annual126.shape[0]):
    for j in np.arange(annual126.shape[1]):
        if annual126[i,j] > 294: annual126[i,j] = 294
glacannual126 = np.hstack([yearsnew,annual126/10])

for i in np.arange(annual245.shape[0]):
    for j in np.arange(annual245.shape[1]):
        if annual245[i,j] > 294: annual245[i,j] = 294
glacannual245 = np.hstack([yearsnew,annual245/10])

for i in np.arange(glac126_2500.shape[0]):
    for j in np.arange(glac126_2500.shape[1]):
        if glac126_2500[i,j] > 294: glac126_2500[i,j] = 294
glac126_2500data = np.hstack([years,glac126_2500/10])

for i in np.arange(glac245_2500.shape[0]):
    for j in np.arange(glac245_2500.shape[1]):
        if glac245_2500[i,j] > 294: glac245_2500[i,j] = 294
glac245_2500data = np.hstack([years,glac245_2500/10])

glac26 = pd.DataFrame(np.vstack((quant,glac126_2500data[199,1:])).T)

glac45 = pd.DataFrame(np.vstack((quant,glac245_2500data[199,1:])).T)


glac_years = np.append(yearsnew,years)
glac_126 = np.concatenate((annual126,glac126_2500))
glac_245 = np.concatenate((annual245, glac245_2500))

glac_126_csv = np.concatenate((np.reshape(glac_years, (481,1)), glac_126/1000),axis=1)
glac_245_csv = np.concatenate((np.reshape(glac_years, (481,1)), glac_245/1000),axis=1)

#np.savetxt('./Created_Data/Glaciers_2500_SSP126_quantiles.csv', glac_126_csv, delimiter=',')
#np.savetxt('./Created_Data/Glaciers_2500_SSP245_quantiles.csv', glac_245_csv, delimiter=',')

Load up all fast track quantiles for LWS, and change baseline to 2020.

In [38]:
path = './Data/GMSL2500_fasttrack.nc'

ds = nc.Dataset(path)

print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: Global mean sea level projections up to 2500
    subtitle: Including individual components and their combination (sum)
    dimensions(sizes): quant(107), years(481), rcp(2)
    variables(dimensions): float64 quant(quant), float64 years(years), float64 rcp(rcp), float64 the(quant, years, rcp), float64 lws(quant, years, rcp), float64 ant(quant, years, rcp), float64 gre(quant, years, rcp), float64 gic(quant, years, rcp), float64 tslc(quant, years, rcp)
    groups: 


In [39]:
lws26 = ds['lws'][:,:,0]
lws26 = lws26 - lws26[:,0:1]
lws45 = ds['lws'][:,:,1]
lws45 = lws45 - lws45[:,0:1]

quant = ds['quant'][:]

years_fast = ds['years'][:]

lws26 = lws26.T
lws45 = lws45.T

lws_26_csv = np.concatenate((np.reshape(years_fast, (481,1)), lws26/100),axis=1)
lws_45_csv = np.concatenate((np.reshape(years_fast, (481,1)), lws45/100),axis=1)

#np.savetxt('./Created_Data/LWS_2500_RCP26_quantiles.csv', lws_26_csv, delimiter=',')
#np.savetxt('./Created_Data/LWS_2500_RCP45_quantiles.csv', lws_45_csv, delimiter=',')

Load up THE data from Victor. Change baseline to 2020, interpolate to annual data.

In [97]:
the26 = pd.read_csv('./Data/TE26.csv', delimiter=';', header = None).T
the45 = pd.read_csv('./Data/TE45.csv', delimiter=';', header = None).T

the_years = np.asarray(pd.read_csv('./Data/ts_te.csv', delimiter=';', header = None))
the_years = the_years.reshape((49,))

the26 = the26 - the26.iloc[0,:]
the45 = the45 - the45.iloc[0,:]

annual_the26 = np.zeros((glac_years.shape[0], the26.shape[1]))
for i in np.arange(the26.shape[1]):
    f26 = interp1d(the_years, the26.iloc[:,i], fill_value="extrapolate")
    annual_the26[:,i] = f26(glac_years)
    
annual_the45 = np.zeros((glac_years.shape[0], the45.shape[1]))
for i in np.arange(the45.shape[1]):
    f45 = interp1d(the_years, the45.iloc[:,i], fill_value="extrapolate")
    annual_the45[:,i] = f45(glac_years)
    
the_26_csv = np.concatenate((np.reshape(glac_years, (481,1)), annual_the26),axis=1)
the_45_csv = np.concatenate((np.reshape(glac_years, (481,1)), annual_the45),axis=1)

np.savetxt('./Created_Data/THE_2500_RCP26_quantiles.csv', the_26_csv, delimiter=',')
np.savetxt('./Created_Data/THE_2500_RCP45_quantiles.csv', the_45_csv, delimiter=',')

Transform all data in simulations into quantiles.

In [98]:
DeConto_26_quant = np.quantile(DeConto_26_2500[20:,:], quant, axis=1).T
DeConto_45_quant = np.quantile(DeConto_45_2500[20:,:], quant, axis=1).T

#np.savetxt('./Created_Data/DeConto_2500_RCP26_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), DeConto_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/DeConto_2500_RCP45_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), DeConto_45_quant),axis=1), delimiter=',')

Bulthuis_26_quant = np.quantile(annual_bult_26[20:,:], quant, axis=1).T
Bulthuis_45_quant = np.quantile(annual_bult_45[20:,:], quant, axis=1).T

#np.savetxt('./Created_Data/Bulthuis_2500_RCP26_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), Bulthuis_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/Bulthuis_2500_RCP45_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), Bulthuis_45_quant),axis=1), delimiter=',')

Lowry_26_quant = np.quantile(Lowry_26_2500[50:,:], quant, axis=1).T
Lowry_45_quant = np.quantile(Lowry_45_2500[50:,:], quant, axis=1).T

#np.savetxt('./Created_Data/Lowry_2500_RCP26_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), Lowry_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/Lowry_2500_RCP45_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), Lowry_45_quant),axis=1), delimiter=',')

Aschwanden_26_quant = np.quantile(SLE26[12:493,1:], quants, axis=1).T
Aschwanden_45_quant = np.quantile(SLE45[12:493,1:], quants, axis=1).T

#np.savetxt('./Created_Data/Aschwanden_2500_RCP26_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)),Aschwanden_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/Aschwanden_2500_RCP45_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)),Aschwanden_45_quant),axis=1), delimiter=',')

LARMIP_26_quant = np.quantile(LAR_126, quants, axis=1).T
LARMIP_45_quant = np.quantile(LAR_245, quants, axis=1).T

#np.savetxt('./Created_Data/LARMIP_2500_SSP126_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)),LARMIP_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/LARMIP_2500_SSP245_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)),LARMIP_45_quant),axis=1), delimiter=',')

GrIS_AR6_26_quant = np.quantile(gris_26/1000, quants, axis=1).T
GrIS_AR6_45_quant = np.quantile(gris_45/1000, quants, axis=1).T

#np.savetxt('./Created_Data/GrIS_AR6_2500_SSP126_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)),GrIS_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/GrIS_AR6_2500_SSP245_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)),GrIS_45_quant),axis=1), delimiter=',')

Glac_26_quant = glac_126_csv[:,1:]
Glac_45_quant = glac_245_csv[:,1:]

THE_26_quant = the_26_csv[:,1:]
THE_45_quant = the_45_csv[:,1:]

LWS_26_quant = lws_26_csv[:,1:]
LWS_45_quant = lws_45_csv[:,1:]

 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:4650: UserWarning:Warning: 'partition' will ignore the 'mask' of the MaskedArray.


Average AIS and GrIS data, and create GMSL quantiles.

In [99]:
GrIS_26_quant = np.mean((Aschwanden_26_quant, GrIS_AR6_26_quant), axis = 0)
GrIS_45_quant = np.mean((Aschwanden_45_quant, GrIS_AR6_45_quant), axis = 0)

#np.savetxt('./Created_Data/GrIS_2500_SSP126_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), GrIS_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/GrIS_2500_SSP245_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), GrIS_45_quant),axis=1), delimiter=',')

AIS_26_quant = np.mean((DeConto_26_quant, Bulthuis_26_quant, Lowry_26_quant, LARMIP_26_quant), axis = 0)
AIS_45_quant = np.mean((DeConto_45_quant, Bulthuis_45_quant, Lowry_45_quant, LARMIP_45_quant), axis = 0)

#np.savetxt('./Created_Data/AIS_2500_RCP26_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), AIS_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/AIS_2500_RCP45_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), AIS_45_quant),axis=1), delimiter=',')

GMSL_26_quant = AIS_26_quant + GrIS_26_quant + Glac_26_quant + THE_26_quant + LWS_26_quant
GMSL_45_quant = AIS_45_quant + GrIS_45_quant + Glac_45_quant + THE_45_quant + LWS_45_quant

#np.savetxt('./Created_Data/GMSL_2500_RCP26_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), GMSL_26_quant),axis=1), delimiter=',')
#np.savetxt('./Created_Data/GMSL_2500_RCP45_quantiles.csv', np.concatenate((np.reshape(years_fast, (481,1)), GMSL_45_quant),axis=1), delimiter=',')
